<a href="https://colab.research.google.com/github/young-hyun-park/capston_design/blob/main/Resnet50_AsymmetricLossd_weight_norm_bn_multilabelclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import numpy as np
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.io import decode_image, read_file
from torchsummary import summary
from PIL import Image
import os
import re
import nibabel as nib

## Data loading

In [4]:
zip_path = sorted(glob('/content/drive/Shareddrives/캡스톤 디자인1/dataset/original_512/*'))

In [6]:
import zipfile
for path in zip_path:
  zip_file = zipfile.ZipFile(path)
  zip_file.extractall('/content/datasets/'+path.split('/')[-1].split('.')[0])
  zip_file.close()

In [7]:
my_glob = glob('/content/datasets/images*/*.png')
print('Number of Observations: ', len(my_glob))

Number of Observations:  76666


In [8]:
df = pd.read_csv('/content/drive/Shareddrives/캡스톤 디자인1/NIH_Dataframe.csv')

In [9]:
full_img_paths = {os.path.basename(x): x for x in my_glob}
df['full_path'] = df['img_ind'].map(full_img_paths.get)

In [10]:
df

,Cardiomegaly,Hernia,Infiltration,Nodule,Emphysema,Effusion,Atelectasis,Pleural_Thickening,Pneumothorax,Mass,Fibrosis,Consolidation,Edema,Pneumonia,img_ind,full_path
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000001_000.png,/content/datasets/images_01/00000001_000.png
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000001_001.png,/content/datasets/images_01/00000001_001.png
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000001_002.png,/content/datasets/images_01/00000001_002.png
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000003_000.png,/content/datasets/images_01/00000003_000.png
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00000003_001.png,/content/datasets/images_01/00000003_001.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51754,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,00030786_007.png,/content/datasets/images_12/00030786_007.png
51755,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,00030789_000.png,/content/datasets/images_12/00030789_000.png
51756,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,00030793_000.png,/content/datasets/images_12/00030793_000.png
51757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,00030795_000.png,/content/datasets/images_12/00030795_000.png


In [11]:
df_for_col = pd.read_csv('/content/drive/Shareddrives/캡스톤 디자인1/NIH_Dataframe.csv')
target_cols = df_for_col.drop(['img_ind'], axis=1).columns.to_list()
n_classes = len(target_cols)

In [12]:
n_classes

14

In [13]:
from sklearn.model_selection import train_test_split
paths = df['full_path'].values
columns = df.columns[:-2]
labels = df[columns].values
(train_path, val_path ,train_labels , val_labels) = train_test_split(paths,labels,test_size = 0.2,random_state = 11)
(train_path, tuning_path, train_labels, tuning_labels) = train_test_split(train_path,train_labels,test_size = 0.12,random_state = 11)

In [14]:
#pip install -U albumentations

In [15]:
!pip install --upgrade albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 102 kB 4.4 MB/s 
     |████████████████████████████████| 47.8 MB 1.7 MB/s 
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [16]:
#!pip uninstall opencv-python-headless

In [17]:
#!pip install opencv-python-headless==4.1.2.30

In [18]:
!pip install --upgrade opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60.5 MB 1.1 MB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


## Data Loading

In [19]:
class Train_Dataset(Dataset):
    def __init__(self, data_path,labels, transform = None):
        self.data_path = data_path
        self.transform = transform
        self.labels = labels
    def __len__(self):
        return len(self.data_path)
    def __getitem__(self,idx):
        path = self.data_path[idx]
        img = Image.open(path)
        img = np.array(img)/255
        img= np.stack((img,)*3, axis=-1)
        label = self.labels[idx]
        if self.transform is not None:
            transformed = self.transform(image=img)
            image = transformed['image']
        return image, label

In [20]:
class Val_Dataset(Dataset):
    def __init__(self, data_path,labels, transform = None):
        self.data_path = data_path
        self.transform = transform
        self.labels = labels
    def __len__(self):
        return len(self.data_path)
    def __getitem__(self,idx):
        path = self.data_path[idx]
        img = Image.open(path)
        img = np.array(img)/255
        img= np.stack((img,)*3, axis=-1)
        label = self.labels[idx]
        if self.transform is not None:
            transformed = self.transform(image=img)
            image = transformed['image']
        return image, label

In [21]:
import albumentations as A                                                                           
from albumentations.pytorch import ToTensorV2

In [22]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

data_transforms = {
    'train': A.Compose(
    [
     A.CenterCrop(384,384),
     A.RandomResizedCrop(224,224),
     A.HorizontalFlip(p=0.5),
     A.Rotate ([-7,7], p = 0.5),
     ToTensorV2()
     ]
    ),
    'val': A.Compose([
                      A.Resize (224,224),
                      ToTensorV2()   
    ]
                           )
}
'''
A.RandomCrop(224, 224),
     A.OneOf([
              A.HorizontalFlip(p=1),
              A.RandomRotate90(p=1),
              A.VerticalFlip(p=1)            
    ], p=1), 
'''

'\nA.RandomCrop(224, 224),\n     A.OneOf([\n              A.HorizontalFlip(p=1),\n              A.RandomRotate90(p=1),\n              A.VerticalFlip(p=1)            \n    ], p=1), \n'

## Model

In [23]:
pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 4.2 MB/s 


#Model pretrain(fc layer)|

In [24]:
import timm

In [25]:
#timm.list_models(pretrained= False)

In [61]:
pretrained_model = timm.create_model('resnet50',pretrained = True, num_classes = 14)

In [62]:
from torch.nn.utils import weight_norm

In [63]:
class Conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, activation=True, **kwargs) -> None:
        super(Conv_block, self).__init__()
        self.relu = nn.ReLU()
        self.conv = weight_norm(nn.Conv2d(in_channels, out_channels, **kwargs)) # kernel size = ...
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.activation = activation

    def forward(self, x):

        if not self.activation:
            return self.batchnorm(self.conv(x))
        
        return self.relu(self.batchnorm(self.conv(x)))

class Res_block(nn.Module):
    def __init__(self, in_channels, red_channels, out_channels, is_plain=False):
        super(Res_block,self).__init__()
        self.relu = nn.ReLU()
        self.is_plain = is_plain
        
        if in_channels==64:
            self.convseq = nn.Sequential(
                                    Conv_block(in_channels, red_channels, kernel_size=1, padding=0),
                                    Conv_block(red_channels, red_channels, kernel_size=3, padding=1),
                                    Conv_block(red_channels, out_channels, activation=False, kernel_size=1, padding=0)
            )
            self.iden = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1)
        elif in_channels == out_channels:
            self.convseq = nn.Sequential(
                                    Conv_block(in_channels, red_channels, kernel_size=1, padding=0),
                                    Conv_block(red_channels, red_channels, kernel_size=3, padding=1),
                                    Conv_block(red_channels, out_channels, activation=False, kernel_size=1, padding=0)
            )
            self.iden = nn.Identity()
        else:
            self.convseq = nn.Sequential(
                                    Conv_block(in_channels, red_channels, kernel_size=1, padding=0, stride=2),
                                    Conv_block(red_channels, red_channels, kernel_size=3, padding=1),
                                    Conv_block(red_channels, out_channels, activation=False, kernel_size=1, padding=0)
                
            )
            self.iden = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2)
        
    def forward(self, x):
        y = self.convseq(x)
        if self.is_plain:
            x = y
        else:
            x = y + self.iden(x)
        x = self.relu(x)  # relu(skip connection)
        return x

In [64]:
class ResNet(nn.Module):
    def __init__(self, in_channels=3 , num_classes=14, is_plain=False):
        self.num_classes = num_classes
        super(ResNet, self).__init__()
        self.conv1 = Conv_block(in_channels=in_channels, out_channels=64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.conv2_x = nn.Sequential(
                                        Res_block(64, 64, 256, is_plain),
                                        Res_block(256, 64, 256, is_plain),
                                        Res_block(256, 64, 256, is_plain)
        )
        
        self.conv3_x = nn.Sequential(
                                        Res_block(256, 128, 512, is_plain),
                                        Res_block(512, 128, 512, is_plain),
                                        Res_block(512, 128, 512, is_plain),
                                        Res_block(512, 128, 512, is_plain)
        )

        self.conv4_x = nn.Sequential(
                                        Res_block(512, 256, 1024, is_plain),
                                        Res_block(1024, 256, 1024, is_plain),
                                        Res_block(1024, 256, 1024, is_plain),
                                        Res_block(1024, 256, 1024, is_plain),
                                        Res_block(1024, 256, 1024, is_plain),
                                        Res_block(1024, 256, 1024, is_plain)
        )
        
        self.conv5_x = nn.Sequential(
                                        Res_block(1024, 512, 2048, is_plain),
                                        Res_block(2048, 512, 2048, is_plain),
                                        Res_block(2048, 512, 2048, is_plain),
        )

        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.fc = nn.Linear(2048,num_classes)

    def forward(self,x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2_x(x)
        x = self.conv3_x(x)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x

In [65]:
model = ResNet()

In [66]:
pretrained_dict = pretrained_model.state_dict()
model_dict = model.state_dict()
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)

<All keys matched successfully>

In [67]:
train_data = Train_Dataset(train_path,train_labels,transform = data_transforms['train'])
val_data = Val_Dataset(val_path,val_labels,transform =  data_transforms['val'])

In [68]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]

# Number of classes in the dataset
num_classes = 14
patience = 7
# Batch size for training (change depending on how much memory you have)
batch_size = 16

# Number of epochs to train for
num_epochs = 5

In [69]:
train_data = Train_Dataset(train_path,train_labels,transform = data_transforms['train'])
val_data = Val_Dataset(val_path,val_labels,transform =  data_transforms['val'])

In [70]:
image_datasets = {'train' : train_data , 'val' : val_data}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True) for x in ['train', 'val']}
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [71]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score , roc_curve , accuracy_score,classification_report,multilabel_confusion_matrix,auc
import tqdm

In [72]:
#gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8,

#Model freeze train fc layer

In [ ]:
from torch.types import Device
import torch.optim as optim
from torch.optim import lr_scheduler
from timm.loss import AsymmetricLossMultiLabel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = model.to(device)
criterion = AsymmetricLossMultiLabel(gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8)

# specify optimizer
optimizer_ft = optim.AdamW(model_ft.parameters(), lr=1e-3)
sgdr_partial = lr_scheduler.CosineAnnealingWarmRestarts (optimizer_ft,T_0 = 50,T_mult = 1,eta_min = 1e-6)

In [ ]:
def train_model(model, dataloaders, criterion,  optimizer, scheduler , num_epochs=25):
    since = time.time()
    train_acc_history = []
    train_loss_hist = [] 
    train_classification_report =[]
    train_confusion_matrix = []
    val_loss_hist = []
    val_acc_history = [] 
    val_classification_report =[]
    val_confusion_matrix = []
    epoch_train_label = []
    epoch_train_output = []
    epoch_val_label = []
    epoch_val_output = []
    counting = 0
    early_stopping = False

    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_loss = 1000000000
    best_train_loss = 100000000
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            start = time.time()
            print('phase : %r' %phase)
            if phase == 'train':
                model.train() # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_acc = 0
            running_precision =0
            running_recall = 0
            running_f1_score =0 
            running_auc = 0
            # Iterate over data.
            full_label = []
            full_output = []
            full_prediction = []
            a= 0
            for inputs, labels in tqdm.notebook.tqdm(dataloaders[phase]):
                a+=1
                inputs = inputs.to(device = device, dtype = torch.float32)
                labels = labels.to(device = device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs,labels.float())
                    pred = nn.Sigmoid()(outputs)
                    full_prediction += pred.detach().cpu().numpy().tolist()
                    pred[pred <0.5] = 0
                    pred[pred >=0.5] = 1
                    if phase == 'train':
                        iters = len(dataloaders[phase])
                    # backward + optimize only if in training phase
                        loss.backward()
                        optimizer.step()
                        scheduler.step(epoch+a/iters)
                # statistics
                running_loss += loss.item() * inputs.size(0)
                full_label+= labels.detach().cpu().numpy().tolist()
                full_output+= pred.detach().cpu().numpy().tolist()
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = accuracy_score(full_label,full_output)
            epoch_classification_report = classification_report(full_label,full_output,target_names = target_cols)
            epoch_multilabel_confusion_matrix = multilabel_confusion_matrix(full_label,full_output)
             
            if phase == 'train':
              train_acc_history.append(epoch_acc)
              train_loss_hist.append(epoch_loss)
              epoch_train_label.append(full_label)
              epoch_train_output.append(full_prediction)
              train_classification_report.append(epoch_classification_report)
              train_confusion_matrix.append(epoch_multilabel_confusion_matrix)
            print('\n{} Loss: {:.4f} Acc: {:.4f}, Time : {:.4f}'.format(phase, epoch_loss ,epoch_acc,time.time()-start))
            #,   , , epoch_precision,epoch_recall, epoch_auc, epoch_f1_score 
            print('\n--------------------------------')
            print('|{}Epoch {}phase multilabel_confusion_matrix report|'.format(epoch+1,phase))
            print('--------------------------------')
            print('\n{}'.format(epoch_multilabel_confusion_matrix))
            print('\n--------------------------------')
            print('\n{}Epoch {}phase classification report'.format(epoch+1,phase))
            print('--------------------------------')
            print(epoch_classification_report)

            # deep copy the model
            if phase == 'val':
              val_acc_history.append(epoch_acc)
              val_loss_hist.append(epoch_loss)
              epoch_val_label.append(full_label)
              epoch_val_output.append(full_prediction)
              val_classification_report.append(epoch_classification_report)
              val_confusion_matrix.append(multilabel_confusion_matrix)

        if phase == 'val' and epoch_loss < best_val_loss:
          best_val_loss = epoch_loss
          best_model_wts = model.state_dict()
          

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best train loss : {:.4f}  Best val loss: {:4f}'.format(best_train_loss ,best_val_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    train_dict = {'Loss' : train_loss_hist , 'Accuarcy' : train_acc_history,'Classification_report' : train_classification_report,'Confusion_matrix' :train_confusion_matrix, 'train_label' : epoch_train_label , 'train_output' : epoch_train_output}
    #
    val_dict = {'Loss' :val_loss_hist , 'Accuarcy' : val_acc_history,'Classification_report' : val_classification_report,'val_label' : epoch_val_label , 'Confusion_matrix' :val_confusion_matrix,'val_output' : epoch_val_output}
    #, , 
    return model, train_dict ,val_dict

In [ ]:

import pickle
# Train and evaluate
start = 1
end = 10
print('\n--------------------------------')
print('|        Epoch {} ~ {}      |'.format(start, end))
print('\n--------------------------------')
model_ft, train_dict, val_dict = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft,sgdr_partial, num_epochs=10)
torch.save(model_ft, '/content/drive/Shareddrives/캡스톤 디자인1/codes/Resnet50_AsymmetricLossd_weight_norm_multilabelclassification_freeze_{}_{}epoch.pt'.format(start, end))
with open('/content/drive/Shareddrives/캡스톤 디자인1/model_history/Resnet50_AsymmetricLossd_weight_norm_multilabelclassification_freeze_{}_{}epoch_train_dict.pkl'.format(start, end),'wb') as fw:
  pickle.dump(train_dict, fw)
with open('/content/drive/Shareddrives/캡스톤 디자인1/model_history/Resnet50_AsymmetricLossd_weight_norm_multilabelclassification_freeze_{}_{}epoch_val_dict.pkl'.format(start, end),'wb') as ff:
  pickle.dump(val_dict, ff)

    
    
'''

confusion maxrix
[[true negative , false positive
false negative , true positive]]

[[tn , fp
fn , tp]]

'''

#Train Model

In [73]:
def train_model(model, dataloaders, criterion,  optimizer, scheduler , num_epochs=25):
    since = time.time()
    train_acc_history = []
    train_loss_hist = [] 
    train_classification_report =[]
    train_confusion_matrix = []
    val_loss_hist = []
    val_acc_history = [] 
    val_classification_report =[]
    val_confusion_matrix = []
    epoch_train_label = []
    epoch_train_output = []
    epoch_val_label = []
    epoch_val_output = []
    counting = 0
    early_stopping = False

    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_loss = 1000000000
    best_train_loss = 100000000
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            start = time.time()
            print('phase : %r' %phase)
            if phase == 'train':
                model.train() # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_acc = 0
            running_precision =0
            running_recall = 0
            running_f1_score =0 
            running_auc = 0
            # Iterate over data.
            full_label = []
            full_output = []
            full_prediction = []
            a= 0
            for inputs, labels in tqdm.notebook.tqdm(dataloaders[phase]):
                a+=1
                inputs = inputs.to(device = device, dtype = torch.float32)
                labels = labels.to(device = device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs,labels.float())
                    pred = nn.Sigmoid()(outputs)
                    full_prediction += pred.detach().cpu().numpy().tolist()
                    pred[pred <0.5] = 0
                    pred[pred >=0.5] = 1
                    if phase == 'train':
                        iters = len(dataloaders[phase])
                    # backward + optimize only if in training phase
                        loss.backward()
                        optimizer.step()
                        scheduler.step(epoch+a/iters)
                # statistics
                running_loss += loss.item() * inputs.size(0)
                full_label+= labels.detach().cpu().numpy().tolist()
                full_output+= pred.detach().cpu().numpy().tolist()
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = accuracy_score(full_label,full_output)
            epoch_classification_report = classification_report(full_label,full_output,target_names = target_cols)
            epoch_multilabel_confusion_matrix = multilabel_confusion_matrix(full_label,full_output)
             
            if phase == 'train':
              train_acc_history.append(epoch_acc)
              train_loss_hist.append(epoch_loss)
              epoch_train_label.append(full_label)
              epoch_train_output.append(full_prediction)
              train_classification_report.append(epoch_classification_report)
              train_confusion_matrix.append(epoch_multilabel_confusion_matrix)
            print('\n{} Loss: {:.4f} Acc: {:.4f}, Time : {:.4f}'.format(phase, epoch_loss ,epoch_acc,time.time()-start))
            #,   , , epoch_precision,epoch_recall, epoch_auc, epoch_f1_score 
            print('\n--------------------------------')
            print('|{}Epoch {}phase multilabel_confusion_matrix report|'.format(epoch+1,phase))
            print('--------------------------------')
            print('\n{}'.format(epoch_multilabel_confusion_matrix))
            print('\n--------------------------------')
            print('\n{}Epoch {}phase classification report'.format(epoch+1,phase))
            print('--------------------------------')
            print(epoch_classification_report)

            # deep copy the model
            if phase == 'val':
              val_acc_history.append(epoch_acc)
              val_loss_hist.append(epoch_loss)
              epoch_val_label.append(full_label)
              epoch_val_output.append(full_prediction)
              val_classification_report.append(epoch_classification_report)
              val_confusion_matrix.append(multilabel_confusion_matrix)

        if phase == 'val' and epoch_loss < best_val_loss:
          best_val_loss = epoch_loss
          best_model_wts = model.state_dict()
          counting = 0
          print('Early Stopping Counter reset')
        else:
            counting += 1
            print('Early Stopping Counter : {}'.format(counting))
        if counting > 9:
          print('Early stopping!')
          early_stopping = True
          break


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best train loss : {:.4f}  Best val loss: {:4f}'.format(best_train_loss ,best_val_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    train_dict = {'Loss' : train_loss_hist , 'Accuarcy' : train_acc_history,'Classification_report' : train_classification_report,'Confusion_matrix' :train_confusion_matrix, 'train_label' : epoch_train_label , 'train_output' : epoch_train_output}
    #
    val_dict = {'Loss' :val_loss_hist , 'Accuarcy' : val_acc_history,'Classification_report' : val_classification_report,'val_label' : epoch_val_label , 'Confusion_matrix' :val_confusion_matrix,'val_output' : epoch_val_output}
    #, , 
    return model, train_dict ,val_dict,early_stopping

In [74]:
from torch.types import Device
import torch.optim as optim
from torch.optim import lr_scheduler
from timm.loss import AsymmetricLossMultiLabel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = AsymmetricLossMultiLabel(gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8)
model_ft = model.to(device)
# specify optimizer
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.0001)
sgdr_partial = lr_scheduler.CosineAnnealingWarmRestarts (optimizer_ft,T_0 = 50,T_mult = 1,eta_min = 0)

In [75]:
for param in model_ft.parameters():
  param.requires_grad = True
  if param.shape[0]== 14:
    break

In [76]:
import pickle
# Train and evaluate
start = 1
end = 50
print('\n--------------------------------')
print('|        Epoch {} ~ {}      |'.format(start, end))
print('\n--------------------------------')
model_ft, train_dict, val_dict  ,early_stopping = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft,sgdr_partial, num_epochs=50)
torch.save(model_ft, '/content/drive/Shareddrives/캡스톤 디자인1/codes/Resnet50_AsymmetricLossd_weight_norm_bn_multilabelclassification_{}_{}epoch.pt'.format(start, end))
with open('/content/drive/Shareddrives/캡스톤 디자인1/model_history/Resnet50_AsymmetricLossd_weight_norm_bn_multilabelclassification_{}_{}epoch_train_dict.pkl'.format(start, end),'wb') as fw:
  pickle.dump(train_dict, fw)
with open('/content/drive/Shareddrives/캡스톤 디자인1/model_history/Resnet50_AsymmetricLossd_weight_norm_bn_multilabelclassification_{}_{}epoch_val_dict.pkl'.format(start, end),'wb') as ff:
  pickle.dump(val_dict, ff)

    
    
'''

confusion maxrix
[[true negative , false positive
false negative , true positive]]

[[tn , fp
fn , tp]]

'''


--------------------------------
|        Epoch 1 ~ 50      |

--------------------------------
Epoch 1/50
----------
phase : 'train'


  0%|          | 0/2278 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



train Loss: 12.1041 Acc: 0.0081, Time : 495.9540

--------------------------------
|1Epoch trainphase multilabel_confusion_matrix report|
--------------------------------

[[[33642   844]
  [ 1891    61]]

 [[36267     5]
  [  166     0]]

 [[  309 22187]
  [  137 13805]]

 [[22930  9052]
  [ 2930  1526]]

 [[34181   516]
  [ 1712    29]]

 [[ 3205 23836]
  [  834  8563]]

 [[ 4988 23249]
  [ 1192  7009]]

 [[32632  1420]
  [ 2275   111]]

 [[26914  5804]
  [ 2908   812]]

 [[24998  7397]
  [ 3036  1007]]

 [[34873   377]
  [ 1167    21]]

 [[28529  4576]
  [ 2704   629]]

 [[33657  1159]
  [ 1486   136]]

 [[35325    92]
  [ 1020     1]]]

--------------------------------

1Epoch trainphase classification report
--------------------------------
                    precision    recall  f1-score   support

      Cardiomegaly       0.07      0.03      0.04      1952
            Hernia       0.00      0.00      0.00       166
      Infiltration       0.38      0.99      0.55     13942
  

  0%|          | 0/647 [00:00<?, ?it/s]

...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



train Loss: 10.6711 Acc: 0.0276, Time : 501.8205

--------------------------------
|45Epoch trainphase multilabel_confusion_matrix report|
--------------------------------

[[[32644  1842]
  [ 1105   847]]

 [[36262    10]
  [  155    11]]

 [[  439 22057]
  [   60 13882]]

 [[23074  8908]
  [ 2153  2303]]

 [[33658  1039]
  [ 1415   326]]

 [[11830 15211]
  [ 1097  8300]]

 [[11324 16913]
  [ 1144  7057]]

 [[32487  1565]
  [ 2028   358]]

 [[27417  5301]
  [ 1738  1982]]

 [[26862  5533]
  [ 2166  1877]]

 [[35067   183]
  [ 1151    37]]

 [[27904  5201]
  [ 2072  1261]]

 [[32599  2217]
  [ 1009   613]]

 [[35417     0]
  [ 1021     0]]]

--------------------------------

45Epoch trainphase classification report
--------------------------------
                    precision    recall  f1-score   support

      Cardiomegaly       0.31      0.43      0.37      1952
            Hernia       0.52      0.07      0.12       166
      Infiltration       0.39      1.00      0.56     13942


  0%|          | 0/647 [00:00<?, ?it/s]


val Loss: 11.4354 Acc: 0.0402, Time : 104.2534

--------------------------------
|45Epoch valphase multilabel_confusion_matrix report|
--------------------------------

[[[ 8389  1391]
  [  241   331]]

 [[10302     2]
  [   47     1]]

 [[    8  6308]
  [    2  4034]]

 [[ 8891   200]
  [ 1174    87]]

 [[ 9759    83]
  [  487    23]]

 [[ 3526  4160]
  [  333  2333]]

 [[ 2491  5619]
  [  216  2026]]

 [[ 9631    41]
  [  677     3]]

 [[ 8556   704]
  [  857   235]]

 [[ 8126  1078]
  [  850   298]]

 [[10013     4]
  [  334     1]]

 [[ 4563  4901]
  [  227   661]]

 [[ 9611   274]
  [  425    42]]

 [[10021    59]
  [  270     2]]]

--------------------------------

45Epoch valphase classification report
--------------------------------
                    precision    recall  f1-score   support

      Cardiomegaly       0.19      0.58      0.29       572
            Hernia       0.33      0.02      0.04        48
      Infiltration       0.39      1.00      0.56      4036
      

'\n\nconfusion maxrix\n[[true negative , false positive\nfalse negative , true positive]]\n\n[[tn , fp\nfn , tp]]\n\n'

#Model Fine Tunning

In [77]:
class Tunning_Dataset(Dataset):
    def __init__(self, data_path,labels, transform = None):
        self.data_path = data_path
        self.transform = transform
        self.labels = labels
    def __len__(self):
        return len(self.data_path)
    def __getitem__(self,idx):
        path = self.data_path[idx]
        img = Image.open(path)
        img = np.array(img)/255
        img= np.stack((img,)*3, axis=-1)
        label = self.labels[idx]
        if self.transform is not None:
            transformed = self.transform(image=img)
            image = transformed['image']
        return image, label

In [78]:

data_transforms = {
    'train': A.Compose(
    [
     A.CenterCrop(384,384),
     A.Resize(224,224),
     A.HorizontalFlip(p=0.5),
     A.Rotate ([-7,7], p = 0.5),
     ToTensorV2()
     ]
    ),
    'val': A.Compose([
                      A.Resize (224,224),
                      ToTensorV2()   
    ]
                           )
}
'''
A.RandomCrop(224, 224),
     A.OneOf([
              A.HorizontalFlip(p=1),
              A.RandomRotate90(p=1),
              A.VerticalFlip(p=1)            
    ], p=1), 
'''

'\nA.RandomCrop(224, 224),\n     A.OneOf([\n              A.HorizontalFlip(p=1),\n              A.RandomRotate90(p=1),\n              A.VerticalFlip(p=1)            \n    ], p=1), \n'

In [79]:
tunning_data = Tunning_Dataset(tuning_path,tuning_labels,transform = data_transforms['train'])
val_data = Val_Dataset(val_path,val_labels,transform =  data_transforms['val'])

In [80]:
image_datasets = {'train' : tunning_data , 'val' : val_data}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True) for x in ['train', 'val']}
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [81]:
from torch.types import Device
import torch.optim as optim
from torch.optim import lr_scheduler
from timm.loss import AsymmetricLossMultiLabel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = AsymmetricLossMultiLabel(gamma_neg=4, gamma_pos=1, clip=0.05, eps=1e-8)
# specify optimizer
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.0001)
sgdr_partial = lr_scheduler.CosineAnnealingWarmRestarts (optimizer_ft,T_0 = 50,T_mult = 1,eta_min = 0)

In [82]:
import pickle
# Train and evaluate
start = 1
end = 50
print('\n--------------------------------')
print('|        Epoch {} ~ {}      |'.format(start, end))
print('\n--------------------------------')
model_ft, train_dict, val_dict  ,early_stopping = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft,sgdr_partial, num_epochs=50)
torch.save(model_ft, '/content/drive/Shareddrives/캡스톤 디자인1/codes/Resnet50_AsymmetricLossd_weight_norm_multilabelclassification_tunning_{}_{}epoch.pt'.format(start, end))
with open('/content/drive/Shareddrives/캡스톤 디자인1/model_history/Resnet50_AsymmetricLossd_weight_norm_multilabelclassification_tunning_{}_{}epoch_train_dict.pkl'.format(start, end),'wb') as fw:
  pickle.dump(train_dict, fw)
with open('/content/drive/Shareddrives/캡스톤 디자인1/model_history/Resnet50_AsymmetricLossd_weight_norm_multilabelclassification_tunning_{}_{}epoch_val_dict.pkl'.format(start, end),'wb') as ff:
  pickle.dump(val_dict, ff)

    
    
'''


confusion maxrix
[[true negative , false positive
false negative , true positive]]

[[tn , fp
fn , tp]]


'''


--------------------------------
|        Epoch 1 ~ 50      |

--------------------------------
Epoch 1/50
----------
phase : 'train'


  0%|          | 0/311 [00:00<?, ?it/s]


train Loss: 10.7361 Acc: 0.0284, Time : 70.6724

--------------------------------
|1Epoch trainphase multilabel_confusion_matrix report|
--------------------------------

[[[4447  270]
  [ 126  126]]

 [[4956    0]
  [  13    0]]

 [[  63 2990]
  [  11 1905]]

 [[3096 1259]
  [ 312  302]]

 [[4495  209]
  [ 219   46]]

 [[2003 1712]
  [ 179 1075]]

 [[1583 2270]
  [ 164  952]]

 [[4431  219]
  [ 275   44]]

 [[3761  718]
  [ 239  251]]

 [[3533  845]
  [ 298  293]]

 [[4778   28]
  [ 162    1]]

 [[3795  728]
  [ 275  171]]

 [[4504  251]
  [ 135   79]]

 [[4831    0]
  [ 138    0]]]

--------------------------------

1Epoch trainphase classification report
--------------------------------
                    precision    recall  f1-score   support

      Cardiomegaly       0.32      0.50      0.39       252
            Hernia       0.00      0.00      0.00        13
      Infiltration       0.39      0.99      0.56      1916
            Nodule       0.19      0.49      0.28       614

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  0%|          | 0/647 [00:00<?, ?it/s]

...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  0%|          | 0/647 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



val Loss: 13.6069 Acc: 0.0888, Time : 85.6641

--------------------------------
|11Epoch valphase multilabel_confusion_matrix report|
--------------------------------

[[[ 7979  1801]
  [  311   261]]

 [[10304     0]
  [   48     0]]

 [[   49  6267]
  [   14  4022]]

 [[ 9052    39]
  [ 1252     9]]

 [[ 9695   147]
  [  477    33]]

 [[ 4759  2927]
  [  742  1924]]

 [[ 4690  3420]
  [  821  1421]]

 [[ 9588    84]
  [  664    16]]

 [[ 9073   187]
  [ 1008    84]]

 [[ 8932   272]
  [ 1068    80]]

 [[ 9991    26]
  [  333     2]]

 [[ 8460  1004]
  [  681   207]]

 [[ 8620  1265]
  [  270   197]]

 [[ 9592   488]
  [  241    31]]]

--------------------------------

11Epoch valphase classification report
--------------------------------
                    precision    recall  f1-score   support

      Cardiomegaly       0.13      0.46      0.20       572
            Hernia       0.00      0.00      0.00        48
      Infiltration       0.39      1.00      0.56      4036
       

'\n\n\nconfusion maxrix\n[[true negative , false positive\nfalse negative , true positive]]\n\n[[tn , fp\nfn , tp]]\n\n\n'